In [1]:
import pandas as pd
train = pd.read_csv("../../data/kaggle/titanic/train.csv")
test = pd.read_csv("../../data/kaggle/titanic/test.csv")
gender_submission = pd.read_csv("../../data/kaggle/titanic/gender_submission.csv")
# ..今いる１つ上のディレクトリ

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
gender_submission.head()
# 性別だけで予測した場合のサンプル提出ファイル

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
print(len(train))
train.isna().sum()

891


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
# 現在の方針
# trainを使用して決定木のモデルを作り、それにtestを入れる。
# 欠損値のある列に対処し、文字の入ってる列を数字で表現。また、Ticketの表記がバラバラなのに対しても対処。

age_median = train["Age"].median()
train["Age"] = train["Age"].fillna(age_median)
# Ageの欠損値を中央値で補完
train = train.dropna(subset=["Embarked"])
# Embarkedはtestの方には欠損が無い為、学習の際には消してしまっても良いと判断
print(len(train))
train.isna().sum()


889


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [8]:
train = train[["Survived","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
# PassengerId,Name,Cabinを排除
# PassengerldとNameは基本、生存率に作用しないと考えたため(NameはMr,Mrs等が影響するかもしれないが)
# Cabinは欠損が多すぎるため

In [9]:
train = pd.get_dummies(train,dtype=int)
train.head()
# 文字列型を数字に変換

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [10]:
del train["Sex_male"]
del train["Embarked_S"]
# 不要な列を削除
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q
0,0,3,22.0,1,0,7.2500,0,0,0
1,1,1,38.0,1,0,71.2833,1,1,0
2,1,3,26.0,0,0,7.9250,1,0,0
3,1,1,35.0,1,0,53.1000,1,0,0
4,0,3,35.0,0,0,8.0500,0,0,0


In [11]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection
# ライブラリのインポート

X = train
y = train["Survived"]
del X["Survived"]
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y,random_state=0)
# Xが教材　ｙが答え　trainが学習用　testが評価用
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train,y_train)

print(model.score(X_test,y_test))
print(model.score(X_train,y_train))
# 評価用での精度が約76%,学習用での精度が約98% → 明らかに過学習/チューニングが必要

0.7623318385650224
0.9834834834834835


In [12]:
model = DecisionTreeClassifier(random_state=0,max_depth=6)
#決定木の深さを指定
model.fit(X_train,y_train)

print(model.score(X_test,y_test))
print(model.score(X_train,y_train))
# 評価用が約77.5%,学習用が約88.5%
# 深さは６階層までにするのが、最もtestでの精度、及び、testとtrainの差が小さかった

0.7757847533632287
0.8858858858858859


In [13]:
# 現在の方針　
# モデルの作成完了/test(実際の予測に使うデータ)が加工前なので、加工

test_raw = test.copy()
# 提出に際し、加工前のtestのPassengerIdが必要になるので、コピーを加工

age_median_test = test_raw["Age"].median()
test_raw["Age"] = test_raw["Age"].fillna(age_median_test)
fare_median_test = test_raw["Fare"].median()
test_raw["Fare"] = test_raw["Fare"].fillna(fare_median_test)
# Age,Fareの欠損値を中央値で補完

test_raw = test_raw[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
# いらない列を削除
# この後に、test = test.reindex(columns=X.columns, fill_value=0)があるので、理論上不要

test_raw = pd.get_dummies(test_raw,dtype=int)
# 文字列型を数字に変換
test_raw = test_raw.reindex(columns=X.columns, fill_value=0)
# testの列をtrainに合わせる
test_raw.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q
0,3,34.5,0,0,7.8292,0,0,1
1,3,47.0,1,0,7.0000,1,0,0
2,2,62.0,0,0,9.6875,0,0,1
3,3,27.0,0,0,8.6625,0,0,0
4,3,22.0,1,1,12.2875,1,0,0


In [14]:
pred = model.predict(test_raw)

In [16]:
submission = pd.DataFrame({"PassengerId": test["PassengerId"],"Survived": pred})
submission.to_csv("../../outputs/kaggle_titanic/submission.csv", index=False)

In [18]:
submission = pd.read_csv("../../outputs/kaggle_titanic/submission.csv")
print(len(submission))

418
